In [2]:
import networkx as nx
import sys
sys.path.append(r'c:\\Users\\rotim\\OneDrive\\Documents\\Reading\\graph-code\\Skoltech-PhD-Thesis\\_codes\\SIR\\Influential-nodes-identification-in-complex-networks-via-information-entropy')
from algorithms import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle

In [3]:
data_file = 'topo'  # 'HepPh'
G = clean_data(data_file)
print(nx.number_of_nodes(G), nx.number_of_edges(G))


34 42


In [4]:
geo_loc_data = get_geo_data("Internet2LatLong.csv")
edge_geo_data_combined = assign_location(G, geo_loc_data)

# set edge attributes
set_edge_attr(G, edge_geo_data_combined)

### Degree, Two Step Neghbor $D - 2SN$

In [20]:
d = [(i, len(n_neighbor(G, i, 1)))
          for (i, j) in sorted(list(G.degree()), key=lambda item: int(item[0]))]
two_sn = [(i, len(n_neighbor(G, i, 2)))
          for (i, j) in sorted(list(G.degree()), key=lambda item: int(item[0]))]
combined_dict_d, combined_dict_k_2 = varying_examples(d, two_sn)

# Get ranked result
opti_rank_m1, ranked_nodes_m1 = rank_result(combined_dict_d, combined_dict_k_2)
opti_rank_m1
# ranked_nodes_m1


[('5', 0.9962640099626401),
 ('12', 0.9672830725462306),
 ('2', 0.9286898839137645),
 ('8', 0.7955251709136109),
 ('29', 0.7955251709136109),
 ('11', 0.6096752816434725),
 ('14', 0.6096752816434725),
 ('16', 0.6096752816434725),
 ('28', 0.6096752816434725),
 ('7', 0.39744499645138387),
 ('19', 0.39744499645138387),
 ('24', 0.39744499645138387),
 ('27', 0.39744499645138387),
 ('6', 0.15278838808250583),
 ('21', 0.15278838808250583)]

### Weighted Degree $WD = (NS/D) - 2SN$

In [21]:
ns_over_d = [(i, sum(hub_information(G, i, 1))/len(n_neighbor(G, i, 1)))
 for (i, j) in sorted(list(G.degree()), key=lambda item: int(item[0]))]

combined_dict, combined_dict_k_2 = varying_examples(ns_over_d, two_sn)

# Get ranked result
opti_rank_m2, ranked_nodes_m2 = rank_result(combined_dict, combined_dict_k_2)
opti_rank_m2
# ranked_nodes_m2


[('27', 0.9844851618495875),
 ('5', 0.7345027474546084),
 ('28', 0.27345263505673767),
 ('12', 0.2674557456648337),
 ('22', 0.029837483220877647)]

### $D - WD$

In [29]:
w_d = [(i[0],i[1]-(j[1])) for i,j in zip(ns_over_d, two_sn)]
combined_dict_w_d, combined_dict_k_w_d = varying_examples(d, w_d)

# Get ranked result
opti_rank_m3, ranked_nodes_m3 = rank_result(combined_dict_w_d, combined_dict_k_w_d)
opti_rank_m3
# ranked_nodes_m3


[('27', 0.9893353941267387),
 ('12', 0.4407547478381754),
 ('5', 0.10787190872467467),
 ('28', 0.07030551260082774)]

In [42]:
infect_prob = compute_probability(G) * 1.5
print(infect_prob)
atio = 1.5
cover_prob = infect_prob / atio
avg = 100
max_iter = 200000


0.8873239436619718


In [43]:
rates = [0.15, 0.3, 0.45, 0.6, 0.75, 0.9]
topk_list = list(map(lambda x: round(nx.number_of_nodes(G) * x), rates))
topk_list


[5, 10, 15, 20, 26, 31]

In [44]:
m1_result = [get_sir_result(G, ranked_nodes_m1, k, avg, infect_prob, cover_prob, max_iter)[-1]/ nx.number_of_nodes(G) for k in rates]
m2_result = [get_sir_result(G, ranked_nodes_m2, k, avg, infect_prob, cover_prob, max_iter)[-1]/ nx.number_of_nodes(G) for k in rates]
m3_result = [get_sir_result(G, ranked_nodes_m3, k, avg, infect_prob, cover_prob, max_iter)[-1]/ nx.number_of_nodes(G) for k in rates]


TypeError: 'float' object cannot be interpreted as an integer